# Lesson 2 - The Riemann zeta function

In this lesson we will go further into programming and learn about some control structures in Python

## Learning Outcomes:
- **Python**
  - Iteration with `for` loops
  - Conditional programming with `if`, `elif`, and `else`
  - The `range` function
  - List comprehensions
- **SageMath**
  - Plotting in 1D

## Mathematical problem

The Riemann zeta function $\zeta(s)$ is defined for $s$ with real part greater than $1$ by 
$$ \zeta(s) = 1 + 2^{-s} + 3^{-s} \cdots = \sum_{n=1}^{\infty} n^{-s} $$
It is known that 
1. $\zeta(s)$ can be extended to an analytic (holomorphic) function in the entire complex plane except for a simple pole with residue $1$ at $s=1$.
2. $\zeta(s)$ satisfies a functional equation when reflecting in the line $\Re(s)=1/2$ of the form: 

$$ \zeta(s) = 2^s \pi^{s-1} \sin(\frac{\pi s}{2}) \Gamma(1-s) \zeta(1-s) $$
3. $\zeta(-2n) = 0$ for each positive integer $n$ (these are called trivial zeros)

One of the main unsolved problems in number theory today is the **Riemann Hypothesis** which states that all non-trivial zeros of $\zeta(s)$ lie on the line $\Re(s) = 1/2$.

The evidence for this conjecture is mainly experimental and has been verified up to a very large height. 

The main aim with the next few sessions is to study the Riemann zeta function and its zeros and (ideally) be able to verify the Riemann Hypothesis up to some height. 

## Python Control Structures

Standard (used in most programming languages):
- Iteration with `for` and `while` loops
- If-then-else statements

More Python specific:
- Generator expressions
- List comprehensions

In [ ]:
for i in range(5): # Iterate over a range of integers using the `range` function. Note that range starts from 0.
    print(i)

The output of the `range` function is an example of a generator expression. It does not actually allocate all elements until needed.

In [ ]:
range(5, 12) # Specify start and end points

We can cast a range to a `list` to see all of its elements.
Note that `range` starts at the left endpoint and stops at the right endpoint without including it!

In [ ]:
list(range(5,12))

Calling `list(range(10^(10^10)))` would run out of memory, but iterating over it is fine (although probably won't finish). <br>
If evaluating the cell below, please call keyboard interrupt or select Kernel -> Interrupt from the toolbar above.

In [ ]:
for i in range(10^(10^10)):
    pass

### Example: `for` loops and list comprehensions
We can evaluate $\zeta(2)$, the zeta function at `s=2`, using a loop and compare to the known value $\frac{\pi}{6}$

In [ ]:
RR.pi()**2 / 6 # The known value we want to calculate

In [ ]:
result = 0 # This cell calculates the partial sum of the first 100 terms of `zeta(2)`
for n in range(1, 100):
    result += n**(-2)
print(result)

We may also use a list comprehension, or directly use the generator expression to compute the partial sum

In [ ]:
sum([n**(-2) for n in range(1, 100)]) # List comprehension

In [ ]:
sum(n**(-2) for n in range(1, 100)) # Generator expression - most memory efficient

### Partial zeta functions
Define the partial zeta functions:
$$
\zeta_{odd} = \sum_{k = 0}^{\infty} (2k + 1)^{-s}, \quad \zeta_{even} = \sum_{k = 0}^{\infty} (2k)^{-s}
$$
**Exercise**: Compute these infinite series for `s=2` up to a certain number of terms

In [ ]:
# Computing an approximations to `zeta_odd(2)`
print(sum([n**(-2) for n in range(1, 100) if n % 2 == 1])) # Using a list comprehension and division by remainder
print(sum(n**(-2) for n in range(1, 100, 2))) # Use `step` keyword argument in `range` function
result = 0
for n in range(1, 100, 2): # Using `range` with `step` size 2 in a `for` loop
    result += n**(-2)
print(result)

In [ ]:
# If we want to compute both even and odd parts simultaneously we can use an if-elif-else:
result_even = 0
result_odd = 0
for n in range(1, 100):
    if n % 2 == 1:
        result_odd += n**(-2)
    elif n % 2 == 0:
        result_even+= n**-2
    else: # This section of code should never run since we've already considered all possibilities for `n`
        pass # Pass does nothing in a loop or function call 
print(f"zeta_odd(2) = {result_odd}")
print(f"zeta_even(2) = {result_even}") # Verify that these sum to `zeta(2)` as calculated above

### Exercise
Write a function with the following specifications:
1. Takes an input a complex number $s$
2. Outputs an approximation of $\zeta(s)$ (of the **same type** as the input) if $\Re(s)>1$.
3. Raises an appropriate error message if $s$ is of the wrong type or not in the correct domain.
4. Has a docstring that explains it

5. (extra): add a parameter to to adjust a desired error estimate. 

In [ ]:
def lesson2_zeta(s, eps=1e-5):
    r"""
    Approximate the Riemann zeta function evaluated at the complex number `s` which must have real part greater than `1`.
    
    Calculated within a certain error bound determined by the optional argument `eps` which defaults to `1e-5`.
    INPUT:
    - ``s`` -- complex number, argument of zeta function
    - ``eps`` -- real number, desired error estimate
    
    OUTPUT: A complex number of the same type (e.g. same precision) as `s`
    """
    # Input checks
    if not isinstance(s, sage.rings.complex_number.ComplexNumber):
        raise ValueError(f"The first argument `s` must be complex number! Received input s = {s}")
    elif not s.real() > 1:
        raise ValueError(f"The input complex number `s` must have real part greater than 1. Received input s = {s}")
    # Compute a crude error bound by estimating the last term
    nmax = ceil(eps**(-1/s.real())) 
    return sum(s.parent()(n)**(-s) for n in range(1, nmax)) # Call to `s.parent()` returns the field containing `s`

In [ ]:
# Compare the output of our function to the builtin SageMath function `zeta`
s = CC(10,1)
err_bound = eps=10**(-10)
result = lesson2_zeta(s, err_bound)
print(result)
print(zeta(s))
print(zeta(s) - result)

### More Jupyter magics
#### The `%run` magic
We can run other notebooks (or Python scripts) from within a Jupyter notebook instance using the `%run` magic, which will run all of the cells in the specified target notebook. <br>
We could use this to import solutions written elsewhere (i.e. in other notebooks or Python files).

However, it is generally not recommended to run external notebooks in this way as it may produce unexpected results, both affecting the state (collection of variable names and values), and possibly repeating a long-slow analysis. For those familiar with Python, this is similar to the reason that using `from <module-name> import *` statements are generally not recommended.

Packaging code into modules and using `import` statements, or executing shorter scripts with the `.py` extension is the preferred way to import external Python code into notebooks.

#### The `%whos` magic
We can view a table of all of the variables which exist in the *interactive namespace* of our current notebook instance, along with their types, and some information about their stored data by using the `%whos` Jupyter magic command.
We can use this to see how the interactive namespace changes before and after we run `ExampleSolutions.ipynb`.

In [ ]:
%whos # See all the variables defined so far, can clear by restarting the kernel

Running the notebook below will define a solution to the exercise above as a function called `zeta_v1`. <br>
This will also execute various other code cells irrelevant to this exercise.

In [ ]:
%run ExampleSolutions.ipynb # May exhibit unexpected behaviour as the file `ExampleSolutions.ipynb` is updated in the future

In [ ]:
%whos # See the additional variables and functions defined in `ExampleSolutions.ipynb`

In [ ]:
zeta_v1(CC(10,1), eps=10**(-10)) # Execute the `zeta_v1` function defined in `ExampleSolutions.ipynb`

The underscore `_` holds the result of the last executed expression or statement in an interactive interpreter session. <br>
Therefore the cell below will display different results depending on the cell you ran previously, including running the same cell multiple times.

See [here](https://stackoverflow.com/questions/5893163/what-is-the-purpose-of-the-single-underscore-variable-in-python) for further information.

In [ ]:
_ - zeta(CC(10,1))

### Plotting in Sagemath with the `plot` function
The `plot()` function in SageMath takes as its first argument a function or list of functions. 
In the case that the passed functions take a single argument, the next two arguments, `start` and `end`, can be used to define the range over which to evaluate and plot them.
TODO:
- Introduce and explain the usage of the SageMath's `latex()` function, which can also be applied to most SageMath objects

In [ ]:
# A first plot of zeta along the horizontal line Im(z) = 1, 2 < Re(z) < 10
p = plot(lambda x: lesson2_zeta(CC(x, 1)).real(), 2, 10)

In [ ]:
p

In [ ]:
latex(p) # Does not work, runs indefinitely, so needs kernel interrupt

### Exercise 
1. Plot the zeta function along the vertical line $\Re(s)=2$, and the horizontal line $\Im(s)=2$ in the same plot
2. Add a legend describing which curve is $\zeta(x+i)$ and which one is $\zeta(2+iy)$

In [ ]:
zeta_on_real_part2 = lambda x: zeta(CC(2,x)).real() # Define lambda functions to evaluate `zeta` along the lines
zeta_on_imag_part1 = lambda x: zeta(CC(x,1)).real()
# The input argument `x` of our passed functions will run from `2` to `10`
plot([zeta_on_real_part2, zeta_on_imag_part1], 2, 10, legend_label=[r'$\zeta(2+xi)$', '$\\zeta(x+i)$'])